In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
def entropy(p1,p2):
    res = -(np.log(p1)*p1 + np.log(p2)*p2)
    return res

In [4]:
print(entropy(0.5,0.5))
print(entropy(0.9,0.1))
print(entropy(.9999,0.0001))

0.6931471805599453
0.3250829733914482
0.0010210290370309323


# binary crossentropy

In [4]:
data = pd.DataFrame(data=[{'true_val':0, 'predicted_proba':0.1}, ## predicted_proba result of sigmoid
                          {'true_val':1, 'predicted_proba':0.7},
                          {'true_val':1, 'predicted_proba':0.4},
                          {'true_val':0, 'predicted_proba':0.2},
                         ])



data['degree_correctness'] = data['true_val']*data['predicted_proba'] + (1-data['true_val'])*(1-data['predicted_proba'])


data['entropy_elem'] = -( data['degree_correctness']*data['degree_correctness'].apply(np.log) 
                         + (1-data['degree_correctness'])*(1-data['degree_correctness']).apply(np.log) ) # entropy_elem = - sum p*log(p)

data['cross_elem'] = - (data['true_val']*data['degree_correctness'].apply(np.log) ### binary cross-entropy_elem = - sum y_true * log(y_proba_pred) 
                        + (1-data['true_val'])*(data['degree_correctness'].apply(np.log))
                       )

data

,true_val,predicted_proba,degree_correctness,entropy_elem,cross_elem
0,0,0.1,0.9,0.325083,0.105361
1,1,0.7,0.7,0.610864,0.356675
2,1,0.4,0.4,0.673012,0.916291
3,0,0.2,0.8,0.500402,0.223144


In [5]:
binary_crossentropy = data['cross_elem'].mean()
binary_crossentropy

np.float64(0.40036743569623084)

In [6]:
def calc_binary_crossentropy(y_true,y_pred_proba):
    degree_correctness = y_true*y_pred_proba + (1-y_true)*(1-y_pred_proba)
    
    cross_elem = -(y_true*degree_correctness.apply(np.log) ## = 0 <=> y_true == 0
                  + (1-y_true)*degree_correctness.apply(np.log) ## = 0 <=> y_true == 1 
                 )
    return cross_elem.mean()
    

In [7]:
calc_binary_crossentropy(data['true_val'],data['predicted_proba'])

np.float64(0.40036743569623084)

# categorical crossentropy

In [8]:
data1 = pd.DataFrame(data= {'true_0':[1,0,0]
                            ,'true_1':[0,0,1]
                            ,'true_2':[0,1,0]
                            
                            ,'pred_0':[0.7,0.1,0.1]
                            ,'pred_1':[0.1,0.2,0.8]
                            ,'pred_2':[0.2,0.7,0.1]}
                    )
display(data1)

,true_0,true_1,true_2,pred_0,pred_1,pred_2
0,1,0,0,0.7,0.1,0.2
1,0,0,1,0.1,0.2,0.7
2,0,1,0,0.1,0.8,0.1


In [9]:
lst_val = [0,1,2]
data1['degree_correctness'] = 0
for n,val in enumerate(lst_val): 
    data1['degree_correctness'] += data1[f'true_{val}']*data1[f'pred_{val}']


data1['cross_elem'] = -data1['degree_correctness'].apply(np.log)

display(data1)

categorical_crossentropy = data1['cross_elem'].mean()
print('categorical_crossentropy=',categorical_crossentropy)

,true_0,true_1,true_2,pred_0,pred_1,pred_2,degree_correctness,cross_elem
0,1,0,0,0.7,0.1,0.2,0.7,0.356675
1,0,0,1,0.1,0.2,0.7,0.7,0.356675
2,0,1,0,0.1,0.8,0.1,0.8,0.223144


categorical_crossentropy= 0.3121644797305582


In [10]:
def calc_categorical_crossentropy(df_y_true, df_y_pred):
    """column names must match"""
    
    vals_true = df_y_true.columns.tolist()
    vals_pred = df_y_pred.columns.tolist()
    if set(vals_true) != set(vals_pred):
        return 'error columns in true!=pred'

    degree_correctness = pd.Series(data= np.zeros(df_y_true.shape[0])
                                    ,index = df_y_true.index
                                  )
    
    for n,val in enumerate(vals_true):
        degree_correctness += df_y_true[val]*df_y_pred[val]
    cross_elem = - degree_correctness.apply(np.log)
    return cross_elem.mean()

In [11]:
df_y_true = data1[['true_0','true_1','true_2']]
df_y_true.columns = [0,1,2]

df_y_pred = data1[['pred_0','pred_1','pred_2']]
df_y_pred.columns = [0,1,2]


In [12]:
calc_categorical_crossentropy(df_y_true=df_y_true,df_y_pred=df_y_pred)

np.float64(0.3121644797305582)

# sparse categorical crossentropy


"sparse_categorical_entropy" utilise les "true_values" = {1,2,3}   
dans tensorflow sparse_categorical_entropy doit encore va faire le get dummies pour nous


In [13]:
data3 = pd.DataFrame(data={'true_val':[0,2,1]
                           ,'pred_0':[0.7,0.1,0.1]
                           ,'pred_1':[0.1,0.2,0.8]
                           ,'pred_2':[0.2,0.7,0.1]}
                    )
data3['id']= data3.index
display(data3)


,true_val,pred_0,pred_1,pred_2,id
0,0,0.7,0.1,0.2,0
1,2,0.1,0.2,0.7,1
2,1,0.1,0.8,0.1,2


In [14]:
pd.get_dummies(data3['true_val'])

,0,1,2
0,True,False,False
1,False,False,True
2,False,True,False


In [15]:
def calc_sparse_categorical_crossentropy(y_true, df_y_pred):
    
    df_y_true = pd.get_dummies(y_true)
    return calc_categorical_crossentropy(df_y_true=df_y_true, df_y_pred=df_y_pred)


In [16]:
df_y_pred = data3[['pred_0','pred_1','pred_2']]
df_y_pred.columns = [0,1,2]

y_true = data3['true_val']

print(calc_sparse_categorical_crossentropy(y_true=y_true, df_y_pred=df_y_pred))

0.3121644797305582
